In [11]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import importlib
import sqlite3

# keeping company information in additional file
import data_file

# Get current year and month
now = datetime.now()

# Subtract one month from current date
last_month = now - relativedelta(months=1)  #

# Format last month as string in YYYYMM format
year_month = last_month.strftime("%Y%m")

print(year_month)

# Construct file path
file_path = "data_files/order22withmu.xlsx"

# Read xlsx file into pandas DataFrame
df = pd.read_excel(file_path)

202307


In [ ]:
#importlib.reload(data_file)

In [12]:
my_company_name = data_file.my_company_name
exclusion_company_one = data_file.exclusion_company_one
exclusion_company_two = data_file.exclusion_company_two

last_column_name = 'order_intake_amount_eur'
df = df.rename(columns={df.columns[-1]: last_column_name})

# necessary modifications of columns

df['bu'] = df['bu'].astype(str)
df['customer_name'] = df['sold_to_customer_n'].str.upper()

# Filter out rows with unwanted value in customer_name
df = df[~df['customer_name'].str.contains(my_company_name) & ~df['customer_name'].str.contains(exclusion_company_one) & ~df['customer_name'].str.contains(exclusion_company_two)]

# Drop rows where 'total' value is 0
df = df.drop(df[df[last_column_name] == 0].index)

# Get 'sales_order_so' where the sum of 'order_intake_amount_eur' is 0
#result = df.groupby('sales_order_so')[last_column_name].transform('sum') == 0
#sales_order_with_sum_zero = df[result]['sales_order_so'].unique().tolist()
#df = df[~df['sales_order_so'].isin(sales_order_with_sum_zero)]

In [13]:
last_year = [202204, 202205, 202206, 202207, 202208, 202209, 202210, 202211,
       202212, 202301, 202302, 202303]

df_last_year = df[df['year_month'].isin(last_year)]

In [14]:
# Group data and calculate the sum of order_intake_amount_eur
grouped_df = df_last_year.groupby('mu_n_latest')['order_intake_amount_eur'].sum().reset_index()

# Save the results to an Excel file
grouped_df.to_excel('data_files/output_file.xlsx', index=False)

In [15]:
conn2 = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers" 
df_customers = pd.read_sql_query(query, conn2)
conn2.close()

In [17]:
df_customers = df_customers[['sold_to_customer', 'customer_name', 'tier_new']]

In [ ]:
df_customers['sold_to_customer'] = df_customers['sold_to_customer'].astype(str)

In [19]:
df['sold_to_customer'] = df['sold_to_customer'].astype(str)

In [20]:
df_total = df.merge(df_customers, how='left', on='sold_to_customer')
# Save the results to an Excel file
df_total.to_excel('data_files/df_total.xlsx', index=False)